# **XGBoost**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install xgboost

In [21]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

In [22]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/AveragePooled/split_features_3s_all_1D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [23]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [24]:
train_data

{'melspectrogram': array([[-0.01764554, -0.97810674, -0.97999614, ..., -0.82831436,
         -0.86151993, -0.9852102 ],
        [-0.0176447 , -0.97808164, -0.9799673 , ..., -0.82414514,
         -0.85394555, -0.982918  ],
        [-0.01764692, -0.97814673, -0.9800371 , ..., -0.81990373,
         -0.8500417 , -0.98001206],
        ...,
        [-0.01758811, -0.57715386, -0.5305197 , ..., -0.86590964,
         -0.8951364 , -0.9996821 ],
        [-0.01772401, -0.58169186, -0.5389059 , ..., -0.8691575 ,
         -0.89791137, -0.99996406],
        [-0.01760773, -0.57361674, -0.5377899 , ..., -0.8711317 ,
         -0.9001475 , -0.99997455]], dtype=float32),
 'mfcc': array([[-1.        , -0.07280675, -0.12869048, ...,  0.02178697,
          0.02212632,  0.04228346],
        [-1.        , -0.08856594, -0.15130392, ...,  0.02659525,
          0.01738   ,  0.04704501],
        [-1.        , -0.10337539, -0.15928885, ...,  0.02455455,
          0.01868342,  0.05395849],
        ...,
        [-0.9

In [25]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [26]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [27]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [28]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [29]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [30]:
display(train_y.shape)
display(train_y[:15])

(13725,)

array([ 7, 12,  4, 17,  3,  9, 14,  3,  0,  0, 10,  2, 12,  3,  1],
      dtype=int32)

In [31]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(13725, 128)

array([-0.0581256 , -0.63822806, -0.6389076 , -0.4953763 , -0.36455613,
       -0.47280535, -0.6928881 , -0.7405781 , -0.7677548 , -0.7374399 ,
       -0.75045246, -0.77124953, -0.7853015 , -0.7958842 , -0.7959697 ,
       -0.81874114, -0.84561497, -0.85084915, -0.8656642 , -0.85950583,
       -0.8056054 , -0.82864517, -0.8715404 , -0.8579188 , -0.87162524,
       -0.8241646 , -0.80134803, -0.8806075 , -0.8931194 , -0.8950114 ,
       -0.89938545, -0.8664572 , -0.90240514, -0.9203411 , -0.9429153 ,
       -0.9389389 , -0.94072586, -0.94264907, -0.95205927, -0.9475349 ,
       -0.9499955 , -0.9482352 , -0.96123224, -0.96147287, -0.9631074 ,
       -0.9561097 , -0.9551953 , -0.94871837, -0.95351607, -0.94429374,
       -0.94507396, -0.93894887, -0.9448011 , -0.94699717, -0.9460722 ,
       -0.94137007, -0.9353987 , -0.9335354 , -0.92789626, -0.92009485,
       -0.9186622 , -0.9323982 , -0.93251014, -0.9347964 , -0.93165   ,
       -0.9263686 , -0.9260132 , -0.9232989 , -0.9198529 , -0.91

'mfcc'

(13725, 20)

...

(13725, 84)

array([-54.3924  , -55.247612, -54.019176, -55.412117, -55.176765,
       -51.641773, -51.608307, -56.090916, -57.880806, -58.957306,
       -57.609154, -57.304836, -56.412144, -55.77131 , -57.49621 ,
       -56.045795, -55.64338 , -56.29201 , -54.53955 , -55.223743,
       -56.586563, -55.49619 , -52.550323, -33.252346, -29.488544,
       -36.211555, -52.73948 , -57.32028 , -59.76588 , -60.917732,
       -62.20523 , -63.252007, -65.78648 , -67.615456, -67.43155 ,
       -65.66191 , -63.76412 , -64.753426, -67.14033 , -66.963356,
       -67.89089 , -69.045784, -68.605064, -69.198074, -70.61911 ,
       -72.82115 , -73.32479 , -74.19051 , -72.36656 , -70.252655,
       -73.62735 , -74.25865 , -72.172646, -70.6866  , -75.13241 ,
       -75.91342 , -75.68041 , -77.47689 , -78.32974 , -78.5558  ,
       -78.68926 , -78.85076 , -78.95114 , -78.8215  , -78.64342 ,
       -78.462425, -78.510155, -78.526566, -78.30219 , -78.121315,
       -77.88373 , -78.19443 , -77.83473 , -77.17658 , -76.501

'id'

(13725,)

'XC214099.mp3'

In [32]:
display(val_y.shape)
display(val_y[:15])

(4231,)

array([12, 18, 12,  1, 17, 16, 19, 16, 17,  8, 11,  4,  1,  9, 10],
      dtype=int32)

In [33]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(4231, 128)

array([-0.01764328, -0.9565721 , -0.93247706, -0.88549644, -0.81249005,
       -0.77785146, -0.76034915, -0.7679908 , -0.7814716 , -0.7643195 ,
       -0.7633203 , -0.76356155, -0.7534367 , -0.7411649 , -0.7278031 ,
       -0.7379763 , -0.74492484, -0.74983066, -0.76243466, -0.76931804,
       -0.78249747, -0.8023202 , -0.8106345 , -0.82639325, -0.8330959 ,
       -0.84451175, -0.849241  , -0.8568743 , -0.86505175, -0.88638127,
       -0.89514667, -0.90498   , -0.9097053 , -0.9165155 , -0.917027  ,
       -0.920565  , -0.9250685 , -0.92276466, -0.9206286 , -0.9269251 ,
       -0.934101  , -0.9314349 , -0.94306   , -0.9392371 , -0.9454409 ,
       -0.9512764 , -0.9590243 , -0.9694562 , -0.97636455, -0.9819564 ,
       -0.98757833, -0.9883887 , -0.99239784, -0.9925326 , -0.9923378 ,
       -0.9928755 , -0.9921703 , -0.98805094, -0.9837086 , -0.98562473,
       -0.9882742 , -0.9882152 , -0.98868024, -0.9876179 , -0.98632133,
       -0.98952055, -0.9845733 , -0.9795667 , -0.98120886, -0.98

'mfcc'

(4231, 20)

...

(4231, 84)

array([-58.28224 , -58.30469 , -60.03862 , -61.265427, -59.94281 ,
       -59.965263, -60.767155, -60.456078, -59.14285 , -58.579327,
       -56.73909 , -57.452988, -59.510807, -58.565063, -58.85057 ,
       -58.601093, -60.21403 , -58.776463, -57.312927, -58.795265,
       -56.85332 , -55.202473, -54.4058  , -53.438473, -52.28848 ,
       -48.160553, -48.251144, -48.59912 , -47.294586, -46.99907 ,
       -46.827312, -46.508904, -48.728092, -48.913437, -48.83963 ,
       -48.400917, -47.28493 , -47.166725, -47.661697, -47.72212 ,
       -47.44012 , -46.3531  , -45.144695, -45.080162, -45.847664,
       -46.067047, -46.776585, -47.623924, -48.124687, -50.001648,
       -51.211334, -52.777264, -53.999992, -55.038715, -55.989574,
       -57.921734, -59.345406, -59.930416, -60.376286, -60.95862 ,
       -61.72118 , -62.10022 , -62.469154, -63.866455, -66.58549 ,
       -67.87702 , -69.35963 , -70.02588 , -69.020294, -68.64077 ,
       -68.66529 , -68.9072  , -69.106926, -68.98829 , -68.596

'id'

(4231,)

'XC383514.mp3'

## **Random Forest Model**

Saving evaluation results

In [34]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [35]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [36]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)

training_features.shape

(13725, 244)

In [37]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)

validation_features.shape

(4231, 244)

Fit the model with training data

In [38]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [39]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3677617584495391


In [40]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.35      0.55      0.43       260
           1       0.41      0.56      0.47       424
           2       0.21      0.43      0.28       200
           3       0.61      0.74      0.67       318
           4       0.08      0.05      0.06        73
           5       0.08      0.04      0.05        99
           6       0.17      0.10      0.13        86
           7       0.44      0.45      0.44       224
           8       0.14      0.52      0.22        50
           9       0.34      0.55      0.42        71
          10       0.24      0.14      0.18       146
          11       0.13      0.01      0.01       549
          12       0.41      0.23      0.30       415
          13       0.54      0.49      0.51       160
          14       0.02      0.04      0.03        46
          15       0.25      0.33      0.29       255
          16       0.95      0.84      0.89   

### **With MFCCs only**

In [41]:
training_features = train_features['mfcc']

training_features.shape

(13725, 20)

In [42]:
validation_features = val_features['mfcc']

validation_features.shape

(4231, 20)

In [43]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [44]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3157645946584732


In [45]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.31      0.62      0.41       260
           1       0.50      0.62      0.55       424
           2       0.16      0.34      0.22       200
           3       0.65      0.76      0.70       318
           4       0.16      0.10      0.12        73
           5       0.00      0.00      0.00        99
           6       0.29      0.24      0.27        86
           7       0.42      0.43      0.43       224
           8       0.17      0.52      0.26        50
           9       0.12      0.41      0.19        71
          10       0.25      0.24      0.24       146
          11       0.62      0.01      0.03       549
          12       0.31      0.26      0.28       415
          13       0.44      0.37      0.40       160
          14       0.04      0.07      0.05        46
          15       0.16      0.30      0.21       255
          16       0.30      0.07      0.12   

### **With Mel-Spectrogram Only**

In [46]:
training_features = train_features['melspectrogram']

training_features.shape

(13725, 128)

In [47]:
validation_features = val_features['melspectrogram']

validation_features.shape

(4231, 128)

In [48]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [49]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.28267549042779483


In [50]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.31      0.43      0.36       260
           1       0.45      0.55      0.50       424
           2       0.33      0.39      0.36       200
           3       0.29      0.64      0.40       318
           4       0.02      0.04      0.03        73
           5       0.25      0.06      0.10        99
           6       0.37      0.26      0.30        86
           7       0.34      0.36      0.35       224
           8       0.10      0.42      0.16        50
           9       0.15      0.41      0.22        71
          10       0.49      0.20      0.28       146
          11       0.00      0.00      0.00       549
          12       0.27      0.17      0.21       415
          13       0.41      0.23      0.29       160
          14       0.01      0.02      0.01        46
          15       0.29      0.42      0.34       255
          16       0.56      0.17      0.26   

## **With Chroma Only**

In [51]:
training_features = train_features['chroma']

training_features.shape

(13725, 12)

In [52]:
validation_features = val_features['chroma']

validation_features.shape

(4231, 12)

In [53]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [54]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 0.9216029143897997
Validation accuracy: 0.20751595367525408


In [55]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.37      0.16       260
           1       0.31      0.36      0.33       424
           2       0.11      0.41      0.17       200
           3       0.67      0.52      0.59       318
           4       0.15      0.04      0.06        73
           5       0.52      0.13      0.21        99
           6       0.00      0.00      0.00        86
           7       0.19      0.25      0.21       224
           8       0.02      0.02      0.02        50
           9       0.14      0.17      0.16        71
          10       0.06      0.03      0.04       146
          11       0.19      0.01      0.02       549
          12       0.23      0.06      0.10       415
          13       0.17      0.07      0.10       160
          14       0.05      0.09      0.06        46
          15       0.22      0.40      0.29       255
          16       0.36      0.24      0.29   

## **With CQT Only**

In [56]:
training_features = train_features['cqt']

training_features.shape

(13725, 84)

In [57]:
validation_features = val_features['cqt']

validation_features.shape

(4231, 84)

In [58]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [59]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23304183408177737


In [60]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.16      0.28      0.20       260
           1       0.50      0.48      0.49       424
           2       0.18      0.32      0.23       200
           3       0.16      0.29      0.21       318
           4       0.00      0.00      0.00        73
           5       0.21      0.04      0.07        99
           6       0.10      0.09      0.10        86
           7       0.47      0.47      0.47       224
           8       0.15      0.38      0.21        50
           9       0.15      0.34      0.21        71
          10       0.13      0.12      0.12       146
          11       0.00      0.00      0.00       549
          12       0.12      0.08      0.10       415
          13       0.25      0.14      0.18       160
          14       0.03      0.11      0.05        46
          15       0.15      0.19      0.17       255
          16       0.67      0.74      0.70   

## **With MFCCs and Mel-Spectrogram**

In [61]:
training_features = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)

training_features.shape

(13725, 148)

In [62]:
validation_features = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)

validation_features.shape

(4231, 148)

In [63]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [64]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.30820137083431814


In [65]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.30      0.54      0.39       260
           1       0.38      0.59      0.46       424
           2       0.20      0.38      0.26       200
           3       0.62      0.74      0.67       318
           4       0.03      0.04      0.04        73
           5       0.09      0.02      0.03        99
           6       0.27      0.09      0.14        86
           7       0.37      0.41      0.38       224
           8       0.14      0.54      0.23        50
           9       0.16      0.41      0.23        71
          10       0.31      0.21      0.25       146
          11       0.00      0.00      0.00       549
          12       0.32      0.17      0.22       415
          13       0.45      0.39      0.42       160
          14       0.02      0.04      0.03        46
          15       0.23      0.33      0.27       255
          16       0.46      0.15      0.22   

## **With MFCCs and Chroma**

In [66]:
training_features = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)

training_features.shape

(13725, 32)

In [67]:
validation_features = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)

validation_features.shape

(4231, 32)

In [68]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [69]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.345072086977074


In [70]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.34      0.67      0.45       260
           1       0.52      0.67      0.58       424
           2       0.16      0.36      0.22       200
           3       0.66      0.78      0.72       318
           4       0.07      0.04      0.05        73
           5       0.00      0.00      0.00        99
           6       0.33      0.22      0.27        86
           7       0.43      0.45      0.44       224
           8       0.18      0.48      0.26        50
           9       0.22      0.41      0.29        71
          10       0.27      0.29      0.28       146
          11       0.28      0.01      0.03       549
          12       0.38      0.27      0.31       415
          13       0.52      0.39      0.44       160
          14       0.01      0.02      0.02        46
          15       0.16      0.35      0.21       255
          16       0.65      0.20      0.31   

## **With MFCCs and CQT**

In [71]:
training_features = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)

training_features.shape

(13725, 104)

In [72]:
validation_features = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 104)

In [73]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [74]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.34223587804301586


In [75]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.32      0.50      0.39       260
           1       0.40      0.51      0.45       424
           2       0.23      0.36      0.28       200
           3       0.57      0.71      0.63       318
           4       0.29      0.10      0.14        73
           5       0.06      0.01      0.02        99
           6       0.23      0.10      0.14        86
           7       0.48      0.50      0.49       224
           8       0.16      0.52      0.24        50
           9       0.25      0.44      0.31        71
          10       0.31      0.36      0.33       146
          11       0.00      0.00      0.00       549
          12       0.37      0.26      0.30       415
          13       0.48      0.28      0.35       160
          14       0.05      0.11      0.07        46
          15       0.14      0.24      0.18       255
          16       0.88      0.81      0.84   

## **With Chroma and CQT**

In [76]:
training_features = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)

training_features.shape

(13725, 96)

In [77]:
validation_features = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 96)

In [78]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [79]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2699125502245332


In [80]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.18      0.33      0.24       260
           1       0.51      0.50      0.50       424
           2       0.17      0.27      0.21       200
           3       0.21      0.37      0.27       318
           4       0.09      0.07      0.08        73
           5       0.21      0.03      0.05        99
           6       0.04      0.05      0.04        86
           7       0.41      0.52      0.46       224
           8       0.15      0.44      0.22        50
           9       0.19      0.39      0.26        71
          10       0.16      0.12      0.14       146
          11       0.00      0.00      0.00       549
          12       0.29      0.16      0.21       415
          13       0.32      0.11      0.16       160
          14       0.04      0.15      0.06        46
          15       0.18      0.24      0.21       255
          16       0.72      0.88      0.79   

## **With Mel-Spectrogram and Chroma**

In [81]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)

training_features.shape

(13725, 140)

In [82]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)

validation_features.shape

(4231, 140)

In [83]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [84]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.29992909477664853


In [85]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.30      0.43      0.36       260
           1       0.45      0.52      0.48       424
           2       0.32      0.43      0.37       200
           3       0.29      0.64      0.40       318
           4       0.01      0.01      0.01        73
           5       0.17      0.08      0.11        99
           6       0.34      0.22      0.27        86
           7       0.38      0.42      0.40       224
           8       0.12      0.44      0.19        50
           9       0.11      0.39      0.17        71
          10       0.47      0.21      0.29       146
          11       0.00      0.00      0.00       549
          12       0.37      0.16      0.22       415
          13       0.55      0.37      0.44       160
          14       0.00      0.00      0.00        46
          15       0.33      0.49      0.39       255
          16       0.55      0.23      0.33   

## **With Mel-Spectrogram and CQT**

In [86]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)

training_features.shape

(13725, 212)

In [87]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)

validation_features.shape

(4231, 212)

In [88]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [89]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3301819900732687


In [90]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.34      0.50      0.41       260
           1       0.44      0.49      0.46       424
           2       0.28      0.41      0.33       200
           3       0.28      0.64      0.39       318
           4       0.01      0.01      0.01        73
           5       0.20      0.09      0.12        99
           6       0.22      0.12      0.15        86
           7       0.52      0.44      0.48       224
           8       0.16      0.52      0.25        50
           9       0.19      0.46      0.27        71
          10       0.39      0.18      0.25       146
          11       0.00      0.00      0.00       549
          12       0.34      0.24      0.28       415
          13       0.38      0.19      0.25       160
          14       0.01      0.02      0.01        46
          15       0.25      0.35      0.29       255
          16       0.82      0.80      0.81   

## **With Mel-Spectrogram, MFCCs, CQT**

In [91]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)

training_features.shape

(13725, 232)

In [92]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)

validation_features.shape

(4231, 232)

In [93]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [94]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.34389033325454976


In [95]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.32      0.53      0.40       260
           1       0.39      0.57      0.46       424
           2       0.23      0.40      0.29       200
           3       0.53      0.73      0.61       318
           4       0.07      0.04      0.05        73
           5       0.12      0.05      0.07        99
           6       0.14      0.08      0.10        86
           7       0.39      0.42      0.41       224
           8       0.15      0.52      0.24        50
           9       0.34      0.46      0.39        71
          10       0.21      0.13      0.16       146
          11       0.04      0.00      0.00       549
          12       0.37      0.25      0.30       415
          13       0.42      0.41      0.41       160
          14       0.03      0.07      0.05        46
          15       0.23      0.26      0.25       255
          16       0.91      0.71      0.80   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [96]:
training_features = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)

training_features.shape

(13725, 160)

In [97]:
validation_features = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)

validation_features.shape

(4231, 160)

In [98]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [99]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.31907350508154103


In [100]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.33      0.55      0.41       260
           1       0.43      0.58      0.50       424
           2       0.18      0.40      0.25       200
           3       0.61      0.76      0.68       318
           4       0.02      0.04      0.03        73
           5       0.07      0.02      0.03        99
           6       0.16      0.07      0.10        86
           7       0.34      0.37      0.35       224
           8       0.12      0.54      0.20        50
           9       0.21      0.46      0.29        71
          10       0.28      0.20      0.23       146
          11       0.05      0.00      0.00       549
          12       0.35      0.21      0.27       415
          13       0.52      0.42      0.47       160
          14       0.00      0.00      0.00        46
          15       0.27      0.38      0.32       255
          16       0.48      0.21      0.30   

# Review results from all models

In [101]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.000,0.368
3,chroma,0.922,0.208
8,chroma_cqt,1.000,0.270
4,cqt,1.000,0.233
2,melspectrogram,1.000,0.283
9,melspectrogram_chroma,1.000,0.300
12,melspectrogram_chroma_mfcc,1.000,0.319
10,melspectrogram_cqt,1.000,0.330
11,melspectrogram_cqt_mfcc,1.000,0.344
1,mfcc,1.000,0.316


In [102]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.343      0.826
3                    chroma     0.193      0.709
8                chroma_cqt     0.246      0.760
4                       cqt     0.215      0.728
2            melspectrogram     0.260      0.760
9     melspectrogram_chroma     0.278      0.778
10       melspectrogram_cqt     0.307      0.754
11  melspectrogram_cqt_mfcc     0.293      0.809
1                      mfcc     0.289      0.795
6               mfcc_chroma     0.321      0.815
7                  mfcc_cqt     0.320      0.801
5       mfcc_melspectrogram     0.279      0.801
